In [13]:
import pandas as pd
import numpy as np
#cleaning
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
# Sentiment Scoring
from textblob import TextBlob
#word cloud
from wordcloud import WordCloud

In [28]:
tweets = pd.read_json('scrap_result.json', lines=True)

In [5]:
tweets.shape

(10000, 28)

In [12]:
print(f'Data tweets dari {tweets.date.min()} sampai dengan {tweets.date.max()}')

Data tweets dari 2022-01-01 00:14:56+00:00 sampai dengan 2022-10-24 09:27:23+00:00


In [14]:
def clean_text(text):  
    pat1 = r'@[^ ]+'                   
    pat2 = r'https?://[A-Za-z0-9./]+'  
    pat3 = r'\'s'                      
    pat4 = r'\#\w+'                     
    pat5 = r'&amp '                     
    pat6 = r'[^A-Za-z\s]'               
    combined_pat = r'|'.join((pat1, pat2,pat3,pat4,pat5, pat6))
    text = re.sub(combined_pat,"",text).lower()
    return text.strip()

In [15]:
tweets.tail()

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
12441,snscrape.modules.twitter.Tweet,https://twitter.com/halimfaiz_/status/14770930...,2022-01-01 01:43:16+00:00,#percumalaporpolisi https://t.co/Gp3ANQbOO8,#percumalaporpolisi twitter.com/geloraco/statu…,1477093030163148803,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,None,NaN,"{'_type': 'snscrape.modules.twitter.Tweet', 'u...",NaN,None,None,{'_type': 'snscrape.modules.twitter.Coordinate...,"{'_type': 'snscrape.modules.twitter.Place', 'f...",[percumalaporpolisi],None
12442,snscrape.modules.twitter.Tweet,https://twitter.com/DakwahGlobal/status/147708...,2022-01-01 01:29:44+00:00,Maasya Allah . . Semakin Taat Syariat .. #Bi...,Maasya Allah . . Semakin Taat Syariat .. #Bi...,1477089626972037123,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,None,NaN,None,NaN,None,None,None,None,"[Bismillah2022Berkah, percumalaporpolisi]",None
12443,snscrape.modules.twitter.Tweet,https://twitter.com/ArioApp/status/14770878163...,2022-01-01 01:22:33+00:00,@tatawberryy Pake #percumalaporpolisi gak nih?,@tatawberryy Pake #percumalaporpolisi gak nih?,1477087816328744961,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,...,None,NaN,None,1.476964e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[percumalaporpolisi],None
12444,snscrape.modules.twitter.Tweet,https://twitter.com/teruntukroman/status/14770...,2022-01-01 01:16:20+00:00,@tubirfess #percumalaporpolisi ga sii,@tubirfess #percumalaporpolisi ga sii,1477086251836526595,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,None,NaN,None,1.476831e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[percumalaporpolisi],None
12445,snscrape.modules.twitter.Tweet,https://twitter.com/baikbaikyabgst/status/1477...,2022-01-01 00:14:56+00:00,@Erlanggapt @andhikasinggih @traderrebahan #pe...,@Erlanggapt @andhikasinggih @traderrebahan #pe...,1477070800360407040,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,None,NaN,None,1.476820e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[percumalaporpolisi],None


In [29]:
tweets = tweets.drop_duplicates(subset=["renderedContent"], keep="first")
tweets["cleaned_tweet"] = tweets["renderedContent"].apply(clean_text)
tweets.reset_index(inplace=True)
tweets.drop("index", axis=1, inplace=True)

In [30]:
print("Running sentiment process")
for row in tweets.itertuples():
    tweet = tweets.at[row[0], 'cleaned_tweet']
#cari sentimen pakai TextBlob
    analysis = TextBlob(tweet)
    analysis = analysis.translate(from_lang='id', to='en')
#Input nilai ke dataframe
    tweets.at[row[0], 'polarity'] = analysis.sentiment[0]
    tweets.at[row[0], 'subjectivity'] = analysis.sentiment[1]
#Buat polaritas dari setiap baris tweet
    if analysis.sentiment[0]>0:
        tweets.at[row[0], 'Sentiment'] = "Positive"
    elif analysis.sentiment[0]<0:
        tweets.at[row[0], 'Sentiment'] = "Negative"
    else:
        tweets.at[row[0], 'Sentiment'] = "Neutral"

Running sentiment process


NotTranslated: Translation API returned and empty response.

In [19]:
tweets = tweets.dropna(subset=["cleaned_tweet"])

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,cleaned_tweet,polarity,subjectivity,Sentiment
0,snscrape.modules.twitter.Tweet,https://twitter.com/Dradxxd/status/15844766383...,2022-10-24 09:27:23+00:00,@fahrisalam #percumalaporpolisi,@fahrisalam #percumalaporpolisi,1584476638301282305,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[percumalaporpolisi],None,,0.0,0.0,Neutral
1,snscrape.modules.twitter.Tweet,https://twitter.com/FachriiiiiR/status/1584472...,2022-10-24 09:11:21+00:00,@himawansugeha95 @nihaqus @rico_rpp21 nah karn...,@himawansugeha95 @nihaqus @rico_rpp21 nah karn...,1584472602030256130,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[percumalaporpolisi],None,nah karna sudah kebanyakan kejadian seperti it...,0.0,0.0,Neutral
2,snscrape.modules.twitter.Tweet,https://twitter.com/nakKampungSini/status/1584...,2022-10-24 08:25:30+00:00,@Taufiq_PD_DKI Sekelas politisi aja tahu #perc...,@Taufiq_PD_DKI Sekelas politisi aja tahu #perc...,1584461065127366656,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,2,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[percumalaporpolisi],None,sekelas politisi aja tahu \nlebih baik lapor ...,0.0,0.0,Neutral
3,snscrape.modules.twitter.Tweet,https://twitter.com/swaradharma/status/1584446...,2022-10-24 07:26:35+00:00,@CNNIndonesia MONEY TALK \njual beli jabatan j...,@CNNIndonesia MONEY TALK \njual beli jabatan j...,1584446239604473856,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[percumalaporpolisi],None,money talk \njual beli jabatan jika masuk poli...,0.0,0.0,Neutral
4,snscrape.modules.twitter.Tweet,https://twitter.com/KimMeccadina/status/158441...,2022-10-24 05:40:47+00:00,@zarazettirazr @HukumDan @kangdede78 @ListyoSi...,@zarazettirazr @HukumDan @kangdede78 @ListyoSi...,1584419612854079488,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[PercumaLaporPolisi],None,kalo pelakunya buzzerrp dan penguasanya msh yg...,0.0,0.0,Neutral
